In [1]:
import pandas as pd
import re
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    NamesExtractor,
    AddrExtractor,
    DatesExtractor,
    MoneyExtractor,
    Doc
)

In [2]:
class NER():
    def __init__(self):
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)
        self.names_extractor = NamesExtractor(self.morph_vocab)
        self.addr_extractor = AddrExtractor(self.morph_vocab)
        self.dates_extractor = DatesExtractor(self.morph_vocab)
        self.money_extractor = MoneyExtractor(self.morph_vocab)
        self.phone_pattern = re.compile(r"\+?\d{1,4}?[-\s]?\(?\d{1,3}?\)?[-\s]?\d{1,4}[-\s]?\d{1,4}[-\s]?\d{1,9}")

    def _extract_phone_numbers(self, text):
        phone_numbers = self.phone_pattern.findall(text)
        
        return phone_numbers

    def get_tags(self, text):
        doc = Doc(text)

        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)
        doc.tag_ner(self.ner_tagger)

        for span in doc.spans:
            span.normalize(self.morph_vocab)
            span.extract_fact(self.names_extractor)

        # LOC, ORG, PER
        dt = {'LOC': [], 'ORG': [], 'PER': []}

        for span in doc.spans:
            dt[span.type].append(span.normal)

        for key in dt:
            dt[key] = list(set(dt[key]))

        # PHONE, MONEY, ADDRES, DATES
        dt['PHONE'] = self._extract_phone_numbers(text)

        dt['MONEY'] = [f"{_.fact.__dict__['amount']} {_.fact.__dict__['currency']}" for _ in self.money_extractor(text)]

        dt['ADDRES'] = [f"{_.fact.__dict__['type']} {_.fact.__dict__['value']}" for _ in self.addr_extractor(text)]

        date = [_.fact.__dict__ for _ in self.dates_extractor(text)]
        new_date = []
        for d in date:
            tmp = []
            for key in ["day", "month", "year"]:
                if d[key] is not None:
                    tmp.append(str(d[key]))
            new_date.append(".".join(tmp))
        dt['DATE'] = new_date

        return dt

In [3]:
ner = NER()

df = pd.read_csv('train_dataset_train.csv', sep=';')
df.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу


In [6]:
text = "'7000 руб Добрый день, подали документы на ежемесячную выплату в связи с рождением (усыновлением)первого ребенка 9 марта 2021<br>Со мной связывались 30.03 и сказали позвонить 11.04 тк это срок исполнения, к сожалению по номеру +73422150562 не отвечают<br>Могу ли я получить результат? Отказано в выплатах или одобрено<br>Ярина Мария Николаевна<br>г. Пермь Индустриальный район<br>+79226404453<br>Заранее спасибо!"
dt = ner.get_tags(text)
dt

{'LOC': ['Пермь', 'Индустриальный район'],
 'ORG': [],
 'PER': ['Ярина Мария Николаевна'],
 'PHONE': ['7000', '2021', '+73422150562', '+79226404453'],
 'MONEY': ['7000 RUB'],
 'ADDRES': ['город Пермь', 'район Индустриальный'],
 'DATE': ['9.3.2021']}